In [6]:
## Import needed packages
import os
import pandas as pd
import numpy as np
import re

from src.text_cleaning import text_process, text_process_new
from tqdm import tqdm

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import joblib
from sklearn.pipeline import Pipeline

pd.set_option('display.max_rows', 100)

In [7]:
# read dataset
df = pd.read_csv('cronjob_2020_2021.csv')

In [8]:
%%time
# filter - take off Liability, keep Transfer and Revenue
# change post_date to get duration for each lead
df = df[(df.transaction_type == 'credit') & (df.b2c_cat_group != 'Liability')]
df['post_date'] = df['post_date'].apply(lambda x: pd.to_datetime(x))
df

,Lead Id,bank_name,account_number,post_date,description,transaction_type,amount,running_balance,yod_cat,b2c_cat,b2c_cat_group,source_type,source_class,biz_keyword
37,14277,Citibank (online.citibank.com) - Bank,5900,2020-01-09,ACH Electronic CreditMBI SETL,credit,208.18,47601.99,Transfers,General Merchandise Sales,Revenue,NaN,NaN,.*ELECTRONI.*
41,14277,Citibank (online.citibank.com) - Bank,5900,2020-02-28,Interest Payment,credit,0.31,13439.47,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*
44,14277,Citibank (online.citibank.com) - Bank,5900,2020-02-14,Safety Check TransferDONOR XXXXXXXX5919,credit,5800.00,27.12,Transfers,Inter-Account Transfers,Transfer,NaN,NaN,.*TRANSFER.*
52,14277,Citibank (online.citibank.com) - Bank,5900,2020-06-30,Interest Payment,credit,0.38,397.80,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*
63,14277,Citibank (online.citibank.com) - Bank,5900,2020-05-29,Interest Payment,credit,0.97,39547.87,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48292042,1277398,Chase,xxxx2375,2021-02-11,ATM CASH DEPOSIT 02/11 795 E LANCASTER AVE VIL...,credit,500.00,91.06,Deposits,Deposit Item,Revenue,NaN,NaN,.*ATM.*DEPOSIT.*
48292043,1277398,Chase,xxxx2375,2021-02-11,QuickPay with Zelle payment from Kendra Kearne...,credit,250.00,-408.94,Transfers,Misc Revenues,Revenue,NaN,NaN,.*ZELLE.*
48292046,1277398,Chase,xxxx2375,2021-02-08,QuickPay with Zelle payment from APRIL BRUNSON...,credit,400.00,401.06,Transfers,Misc Revenues,Revenue,NaN,NaN,.*ZELLE.*
48292047,1277398,Chase,xxxx2375,2021-02-02,Cash App*Cash Out VISA DIRECT CA XX8554 0...,credit,15.75,1.06,Other Income,Misc Revenues,Revenue,NaN,NaN,.*CASH.*APP.*


In [9]:
df.shape

(11762325, 14)

In [10]:
len(df.bank_name.unique())

2008

In [11]:
df.bank_name.value_counts()[:50].to_frame()

,bank_name
bank of america,2451845
Chase,2419299
Wells Fargo,1571789
Navy Federal Credit Union,745033
PNC Bank,320091
Regions Bank,316494
Wells Fargo Bank,276618
Capital One,229268
TD Bank,188166
Wells Fargo - Bank,164629


In [12]:
# copy of original dataframe

In [13]:
%%time
df_copy = df.copy()

Wall time: 501 ms


## Remove lead with less than 3 month

In [14]:
def Remove_Lead(dataframe):
    lead_df = df.groupby('Lead Id').agg({'bank_name': ['unique', 'nunique'],'running_balance':'count', 
                                    'post_date':['min', 'max']})
    lead_df.columns = lead_df.columns.droplevel()
    
    lead_df['duration'] = (lead_df['max'] - lead_df['min']) / np.timedelta64(1, 'M')
    lead_df['duration'] = lead_df['duration'].astype(int)
    lead_df.reset_index(inplace=True)
    lead_df = lead_df[lead_df.duration > 3]
    
    print('unique lead:', len(lead_df['Lead Id'].unique()))
    print('total transac:', lead_df['count'].sum())
    return lead_df

In [15]:
%%time
lead_df = Remove_Lead(df_copy)
lead_df

unique lead: 62457
total transac: 10969209
Wall time: 6.43 s


,Lead Id,unique,nunique,count,min,max,duration
0,14277,[Citibank (online.citibank.com) - Bank],1,39,2020-01-09,2020-10-30,9
2,19218,[TD Bank],1,21,2020-09-14,2021-04-30,7
3,21138,[BBVA Compass],1,153,2020-01-02,2020-08-10,7
4,22378,[bank of america],1,878,2020-02-05,2021-10-22,20
5,23185,[Chase],1,1014,2020-01-02,2021-04-16,15
...,...,...,...,...,...,...,...
86029,1277215,[Chase],1,149,2021-02-02,2022-01-03,11
86031,1277287,[Navy Federal Credit Union],1,134,2021-07-01,2022-01-04,6
86032,1277324,[Chase],1,243,2021-02-03,2021-12-20,10
86034,1277369,[CitiBusiness Online],1,476,2021-07-08,2022-01-03,5


In [16]:
df_new = df_copy[df_copy['Lead Id'].isin(list(lead_df['Lead Id'].unique()))]
df_new.reset_index(inplace=True, drop=True)
df_new

,Lead Id,bank_name,account_number,post_date,description,transaction_type,amount,running_balance,yod_cat,b2c_cat,b2c_cat_group,source_type,source_class,biz_keyword
0,14277,Citibank (online.citibank.com) - Bank,5900,2020-01-09,ACH Electronic CreditMBI SETL,credit,208.18,47601.99,Transfers,General Merchandise Sales,Revenue,NaN,NaN,.*ELECTRONI.*
1,14277,Citibank (online.citibank.com) - Bank,5900,2020-02-28,Interest Payment,credit,0.31,13439.47,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*
2,14277,Citibank (online.citibank.com) - Bank,5900,2020-02-14,Safety Check TransferDONOR XXXXXXXX5919,credit,5800.00,27.12,Transfers,Inter-Account Transfers,Transfer,NaN,NaN,.*TRANSFER.*
3,14277,Citibank (online.citibank.com) - Bank,5900,2020-06-30,Interest Payment,credit,0.38,397.80,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*
4,14277,Citibank (online.citibank.com) - Bank,5900,2020-05-29,Interest Payment,credit,0.97,39547.87,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969204,1277398,Chase,xxxx2375,2021-02-11,ATM CASH DEPOSIT 02/11 795 E LANCASTER AVE VIL...,credit,500.00,91.06,Deposits,Deposit Item,Revenue,NaN,NaN,.*ATM.*DEPOSIT.*
10969205,1277398,Chase,xxxx2375,2021-02-11,QuickPay with Zelle payment from Kendra Kearne...,credit,250.00,-408.94,Transfers,Misc Revenues,Revenue,NaN,NaN,.*ZELLE.*
10969206,1277398,Chase,xxxx2375,2021-02-08,QuickPay with Zelle payment from APRIL BRUNSON...,credit,400.00,401.06,Transfers,Misc Revenues,Revenue,NaN,NaN,.*ZELLE.*
10969207,1277398,Chase,xxxx2375,2021-02-02,Cash App*Cash Out VISA DIRECT CA XX8554 0...,credit,15.75,1.06,Other Income,Misc Revenues,Revenue,NaN,NaN,.*CASH.*APP.*


In [17]:
len(df_new.bank_name.unique())

1466

In [18]:
df_new.shape

(10969209, 14)

## Model Applied on Large Dataset

In [19]:
%%time

tqdm.pandas() 

def Preprocess(dataframe):
    # clean description, standardize entity name
    dataframe['cleaned_description']  = dataframe['description'].progress_apply(lambda x: text_process_new(x))
    dataframe['cleaned_description'] = dataframe['cleaned_description'].progress_apply(lambda x: ' '.join(x.split()))
    
    return dataframe

Wall time: 1e+03 µs


In [25]:
%%time
def Apply_Model(dataframe):
    # use saved script to predict entity
    pipeline_entity = joblib.load('final_entity_model.pkl')
    dataframe['entity_prediction'] = pipeline_entity.predict(dataframe['cleaned_description'])
    
    return dataframe

Wall time: 0 ns


In [21]:
%%time
# preprocess the description
df_new = Preprocess(df_new)
df_new

100%|██████████| 10969209/10969209 [00:12<00:00, 851499.50it/s]


Wall time: 4min 21s


,Lead Id,bank_name,account_number,post_date,description,transaction_type,amount,running_balance,yod_cat,b2c_cat,b2c_cat_group,source_type,source_class,biz_keyword,cleaned_description
0,14277,Citibank (online.citibank.com) - Bank,5900,2020-01-09,ACH Electronic CreditMBI SETL,credit,208.18,47601.99,Transfers,General Merchandise Sales,Revenue,NaN,NaN,.*ELECTRONI.*,mbi setl
1,14277,Citibank (online.citibank.com) - Bank,5900,2020-02-28,Interest Payment,credit,0.31,13439.47,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*,interest payment
2,14277,Citibank (online.citibank.com) - Bank,5900,2020-02-14,Safety Check TransferDONOR XXXXXXXX5919,credit,5800.00,27.12,Transfers,Inter-Account Transfers,Transfer,NaN,NaN,.*TRANSFER.*,safety check transferdonor
3,14277,Citibank (online.citibank.com) - Bank,5900,2020-06-30,Interest Payment,credit,0.38,397.80,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*,interest payment
4,14277,Citibank (online.citibank.com) - Bank,5900,2020-05-29,Interest Payment,credit,0.97,39547.87,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*,interest payment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969204,1277398,Chase,xxxx2375,2021-02-11,ATM CASH DEPOSIT 02/11 795 E LANCASTER AVE VIL...,credit,500.00,91.06,Deposits,Deposit Item,Revenue,NaN,NaN,.*ATM.*DEPOSIT.*,atm cash deposit lancaster ave villanova pa
10969205,1277398,Chase,xxxx2375,2021-02-11,QuickPay with Zelle payment from Kendra Kearne...,credit,250.00,-408.94,Transfers,Misc Revenues,Revenue,NaN,NaN,.*ZELLE.*,quickpay with zelle payment from kendra kearne...
10969206,1277398,Chase,xxxx2375,2021-02-08,QuickPay with Zelle payment from APRIL BRUNSON...,credit,400.00,401.06,Transfers,Misc Revenues,Revenue,NaN,NaN,.*ZELLE.*,quickpay with zelle payment from april brunson...
10969207,1277398,Chase,xxxx2375,2021-02-02,Cash App*Cash Out VISA DIRECT CA XX8554 0...,credit,15.75,1.06,Other Income,Misc Revenues,Revenue,NaN,NaN,.*CASH.*APP.*,cash app cash out visa direct ca


In [26]:
%%time
# apply saved model on the dataset, get entity_prediction
df_new = Apply_Model(df_new)
df_new

Wall time: 1min 55s


,Lead Id,bank_name,account_number,post_date,description,transaction_type,amount,running_balance,yod_cat,b2c_cat,b2c_cat_group,source_type,source_class,biz_keyword,cleaned_description,entity_prediction
0,14277,Citibank (online.citibank.com) - Bank,5900,2020-01-09,ACH Electronic CreditMBI SETL,credit,208.18,47601.99,Transfers,General Merchandise Sales,Revenue,NaN,NaN,.*ELECTRONI.*,mbi setl,Non Entity
1,14277,Citibank (online.citibank.com) - Bank,5900,2020-02-28,Interest Payment,credit,0.31,13439.47,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*,interest payment,Non Entity
2,14277,Citibank (online.citibank.com) - Bank,5900,2020-02-14,Safety Check TransferDONOR XXXXXXXX5919,credit,5800.00,27.12,Transfers,Inter-Account Transfers,Transfer,NaN,NaN,.*TRANSFER.*,safety check transferdonor,Non Entity
3,14277,Citibank (online.citibank.com) - Bank,5900,2020-06-30,Interest Payment,credit,0.38,397.80,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*,interest payment,Non Entity
4,14277,Citibank (online.citibank.com) - Bank,5900,2020-05-29,Interest Payment,credit,0.97,39547.87,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*,interest payment,Non Entity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969204,1277398,Chase,xxxx2375,2021-02-11,ATM CASH DEPOSIT 02/11 795 E LANCASTER AVE VIL...,credit,500.00,91.06,Deposits,Deposit Item,Revenue,NaN,NaN,.*ATM.*DEPOSIT.*,atm cash deposit lancaster ave villanova pa,Non Entity
10969205,1277398,Chase,xxxx2375,2021-02-11,QuickPay with Zelle payment from Kendra Kearne...,credit,250.00,-408.94,Transfers,Misc Revenues,Revenue,NaN,NaN,.*ZELLE.*,quickpay with zelle payment from kendra kearne...,Non Entity
10969206,1277398,Chase,xxxx2375,2021-02-08,QuickPay with Zelle payment from APRIL BRUNSON...,credit,400.00,401.06,Transfers,Misc Revenues,Revenue,NaN,NaN,.*ZELLE.*,quickpay with zelle payment from april brunson...,Non Entity
10969207,1277398,Chase,xxxx2375,2021-02-02,Cash App*Cash Out VISA DIRECT CA XX8554 0...,credit,15.75,1.06,Other Income,Misc Revenues,Revenue,NaN,NaN,.*CASH.*APP.*,cash app cash out visa direct ca,Cash App


In [35]:
df_new.groupby("Lead Id").agg(agg_amount=("amount","sum"))

,agg_amount
Lead Id,
14277,557997.28
19218,24201.72
21138,113820.68
22378,343040.39
23185,1620265.04
...,...
1277215,173837.10
1277287,41900.19
1277324,84619.15


In [36]:
df_new

,Lead Id,bank_name,account_number,post_date,description,transaction_type,amount,running_balance,yod_cat,b2c_cat,b2c_cat_group,source_type,source_class,biz_keyword,cleaned_description,entity_prediction
0,14277,Citibank (online.citibank.com) - Bank,5900,2020-01-09,ACH Electronic CreditMBI SETL,credit,208.18,47601.99,Transfers,General Merchandise Sales,Revenue,NaN,NaN,.*ELECTRONI.*,mbi setl,Non Entity
1,14277,Citibank (online.citibank.com) - Bank,5900,2020-02-28,Interest Payment,credit,0.31,13439.47,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*,interest payment,Non Entity
2,14277,Citibank (online.citibank.com) - Bank,5900,2020-02-14,Safety Check TransferDONOR XXXXXXXX5919,credit,5800.00,27.12,Transfers,Inter-Account Transfers,Transfer,NaN,NaN,.*TRANSFER.*,safety check transferdonor,Non Entity
3,14277,Citibank (online.citibank.com) - Bank,5900,2020-06-30,Interest Payment,credit,0.38,397.80,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*,interest payment,Non Entity
4,14277,Citibank (online.citibank.com) - Bank,5900,2020-05-29,Interest Payment,credit,0.97,39547.87,Interest,Interest Income,Revenue,NaN,NaN,.*INTEREST.*,interest payment,Non Entity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969204,1277398,Chase,xxxx2375,2021-02-11,ATM CASH DEPOSIT 02/11 795 E LANCASTER AVE VIL...,credit,500.00,91.06,Deposits,Deposit Item,Revenue,NaN,NaN,.*ATM.*DEPOSIT.*,atm cash deposit lancaster ave villanova pa,Non Entity
10969205,1277398,Chase,xxxx2375,2021-02-11,QuickPay with Zelle payment from Kendra Kearne...,credit,250.00,-408.94,Transfers,Misc Revenues,Revenue,NaN,NaN,.*ZELLE.*,quickpay with zelle payment from kendra kearne...,Non Entity
10969206,1277398,Chase,xxxx2375,2021-02-08,QuickPay with Zelle payment from APRIL BRUNSON...,credit,400.00,401.06,Transfers,Misc Revenues,Revenue,NaN,NaN,.*ZELLE.*,quickpay with zelle payment from april brunson...,Non Entity
10969207,1277398,Chase,xxxx2375,2021-02-02,Cash App*Cash Out VISA DIRECT CA XX8554 0...,credit,15.75,1.06,Other Income,Misc Revenues,Revenue,NaN,NaN,.*CASH.*APP.*,cash app cash out visa direct ca,Cash App


In [27]:
df_new.entity_prediction.value_counts().to_frame()

,entity_prediction
Non Entity,8441920
Merchant Service,611910
Cash App,545115
Square,409398
Paypal,181113
Stripe,115325
Intuit,66218
Heartland,65919
Paymentech,52570
Boa Merchant Service,51163


In [29]:
df_new.to_csv("entity_lead_depth_flag.csv")